In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!rm -r data/
!rm -r __MACOSX/
!rm -r test/
!rm -r train/
!rm -r valid/
!rm -r pipeline_output//
!rm data.*
!rm README.*

rm: cannot remove 'data/': No such file or directory
rm: cannot remove '__MACOSX/': No such file or directory
rm: cannot remove 'test/': No such file or directory
rm: cannot remove 'train/': No such file or directory
rm: cannot remove 'valid/': No such file or directory
rm: cannot remove 'pipeline_output/': No such file or directory
rm: cannot remove 'data.*': No such file or directory
rm: cannot remove 'README.*': No such file or directory


In [3]:
!mkdir data
!cp drive/MyDrive/FYP/10000_dataset_binary/data.zip ./data/data.zip
!cp drive/MyDrive/FYP/10000_dataset_binary/*.pt ./

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00


In [5]:
!unzip ./data/data.zip -d data/

串流輸出內容已截斷至最後 5000 行。
  inflating: data/train/labels/S__18907240_jpg.rf.ac08b352cc786a27aaed474a674c4f34.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.bb141be35608d374472e4fc3ab6f300f.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.bf59ec555da15264da0afd5b8260a229.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.c49471ce5d26ced71488163e8e44fba1.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.c4f7a3ce29b3299c5c213515acda71c0.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.c75e720fe90fcc9389be04ea0d80b993.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.c8b26868af001914a5e3a23f314b483e.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.ca42f5e594299709ec0b23f67e9fac2c.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.cef275e74e72e1fd366bbf51e0c8bc1c.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.cf8a9e0fc4c8ed84171084f208298b76.txt  
  inflating: data/train/labels/S__18907240_jpg.rf.d1c8303be602ea0e6317e5f85c45b8bf.txt  


In [6]:
# pipeline_tile_detect_classify_yolo.py
# ------------------------------------------------------------
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path
import json

def tileDetection(path):
    # --------------------------- CONFIG ---------------------------
    DETECTOR_PATH   = "MJ_tile_detection_yolo11.pt"                 # detection .pt
    CLASSIFIER_PATH = "MJ_tile_detection_classification_yolo11.pt" # classification .pt
    IMAGE_PATH      = path                     # glob works
    SAVE_DIR        = Path("pipeline_output")
    SAVE_DIR.mkdir(exist_ok=True)

    DET_CONF = 0.5   # detection confidence
    CLS_CONF = 0.5   # classification confidence (applied inside classifier)

    # --------------------------- MODELS ---------------------------
    detector   = YOLO(DETECTOR_PATH)      # task = 'detect'
    classifier = YOLO(CLASSIFIER_PATH)    # task = 'classify'

    print("Classifier classes:", classifier.names)   # sanity check

    # --------------------------- PIPELINE ------------------------
    results = detector(IMAGE_PATH, imgsz=1920)   # list[Results]

    all_results = []

    for r in results:                                   # one Results per image
        img      = r.orig_img.copy()                    # BGR, original size
        img_path = Path(r.path)
        img_name = img_path.stem

        detections = []

        # ---- Step 1: Detect tiles ----
        for i, box in enumerate(r.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            det_conf = box.conf.item()

            # Crop the tile
            crop = img[y1:y2, x1:x2]
            if crop.size == 0:
                continue

            # --- Step 2: Classify crop ---
            cls_res = classifier(crop, conf=CLS_CONF, verbose=False)[0]

            if cls_res.probs is None:
                tile_type = "unknown"
                cls_conf = 0.0
            else:
                tile_type = cls_res.names[cls_res.probs.top1]
                cls_conf  = cls_res.probs.top1conf.item()

            # Save crop
            # crop_name = f"{img_name}__tile_{i:02d}__{tile_type}__{cls_conf:.3f}.jpg"
            # cv2.imwrite(str(SAVE_DIR / crop_name), crop)

            # ---- Draw on original image ----
            color = (0, 255, 0) if not tile_type == "unknown" else (0, 0, 255)
            label = f"{tile_type} {cls_conf:.2f}"
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # ---- Store result ----
            detections.append({
                "bbox":      [x1, y1, x2, y2],
                "det_conf":  round(det_conf, 4),
                "tile_type": tile_type,
                "cls_conf":  round(cls_conf, 4),
                # "crop":      crop_name
            })

        # ---- Save annotated full image ----
        annotated_path = SAVE_DIR / f"{img_name}__annotated.jpg"
        cv2.imwrite(str(annotated_path), img)

        all_results.append({
            "image":     str(img_path),
            "annotated": str(annotated_path.name),
            "detections": detections
        })

    # --------------------------- SAVE JSON ------------------------
    with open(SAVE_DIR / "results.json", "w") as f:
        json.dump(all_results, f, indent=2)

    print("\nPipeline complete!")
    print(f"  -> {sum(len(r['detections']) for r in all_results)} tiles detected & classified")
    print(f"  -> Results saved to: {SAVE_DIR}")

tileDetection("data/test/images/*.jpg")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Classifier classes: {0: '1m', 1: '1p', 2: '1s', 3: '1z', 4: '2m', 5: '2p', 6: '2s', 7: '2z', 8: '3m', 9: '3p', 10: '3s', 11: '3z', 12: '4m', 13: '4p', 14: '4s', 15: '4z', 16: '5m', 17: '5p', 18: '5s', 19: '5z', 20: '6m', 21: '6p', 22: '6s', 23: '6z', 24: '7m', 25: '7p', 26: '7s', 27: '7z', 28: '8m', 29: '8p', 30: '8s', 31: '9m', 32: '9p', 33: '9s'}

image 1/739 /content/data/test/images/08-3_jpg.rf.1d2e1fd59a8062bd65c08da9a0c6d1a6.jpg: 1920x1920 39 tiles, 74.4ms
image 2/739 /content/data/test/images/10-3_jpg.rf.1019a21ed314ac7bf6026b13060b59ca.jpg: 1920x1920 10 tiles, 74.7ms
image 3/739 /content/data/test/images/202205313_jpg.rf.38f861fa43c15bf4bef7fc54e8746a6f.jpg: 1920x1920 13 t

In [7]:
!zip -r predict.zip ./pipeline_output/

  adding: pipeline_output/ (stored 0%)
  adding: pipeline_output/QQ-20230112181616_png_jpg.rf.4d2bf8a31e1c35ae783478739120d4b3__annotated.jpg (deflated 8%)
  adding: pipeline_output/S__18899057_jpg.rf.fc415d74bf186891d9ff5f601a221b67__annotated.jpg (deflated 5%)
  adding: pipeline_output/IMG_0134_JPG_jpg.rf.3c886a118db045957327e424cf7804f0__annotated.jpg (deflated 5%)
  adding: pipeline_output/S__18907258_jpg.rf.543820ccadcb3111db5a9b2075aa4f7f__annotated.jpg (deflated 4%)
  adding: pipeline_output/S__18899070_jpg.rf.41a9d00ea00ce37016cdb48251f44f50__annotated.jpg (deflated 5%)
  adding: pipeline_output/S__18898990_jpg.rf.1a8565f22677025b2a90bef62aa69daa__annotated.jpg (deflated 6%)
  adding: pipeline_output/S__18907170_jpg.rf.562e05ad2fd60e69a36817d85ae63065__annotated.jpg (deflated 3%)
  adding: pipeline_output/S__18907239_jpg.rf.8fd2348daa368fd8d278ca9fa77f3d23__annotated.jpg (deflated 6%)
  adding: pipeline_output/S__18898975_jpg.rf.580cfff96bee57098996aebc8c4f2dbb__annotated.jpg (

In [8]:
!cp predict.zip  drive/MyDrive/FYP/10000_dataset_binary/